In [1]:
import json
import pandas as pd
from mt_utils import *


option_urls = [
    
    #"https://www.dropbox.com/s/5uc9bannjgtk907/option_bars_0_2_9970.json?dl=1"
    
    #"https://www.dropbox.com/s/9gcwes67twqrg0a/option_bars_0_300_382809.json?dl=1",
    #"https://www.dropbox.com/s/jqoqkibpkp2wskg/option_bars_300_300_482247.json?dl=1"
    
    #"https://www.dropbox.com/s/pfiii5osyg24k0v/options_cache_all.json?dl=1"
    "https://www.dropbox.com/s/2za6imjbn1i28p8/option_bars.json?dl=1"
]

underlying_urls = [
    #"https://www.dropbox.com/s/j69c6wtmzi2i0t9/underlying_bars_0_2_4985.json?dl=1"
    
    #"https://www.dropbox.com/s/itc1e630hjnboqn/underlying_bars_0_300_184827.json?dl=1",
    #"https://www.dropbox.com/s/iijvgunhwnkt0eq/underlying_bars_300_300_234386.json?dl=1"
    
    "https://www.dropbox.com/s/ll8xa6xb1xhrhv7/underlying_bars.json?dl=1"
]

limit_lo = 0
limit_len = 3000

opts1 = None
opts2 = None

QB = False
FIND_MISSING = False  # True = allow searching QC  False = use only cache data

REQUEST_MARKER = '_' # used to prevent implicit symbol conversion to long symbols

#NotebookApp.iopub_data_rate_limit=10000000.0
#NotebookApp.rate_limit_window=3.0


if not QB:
    from datetime import datetime
    from datetime import timedelta
    import requests
    class QuantBook():
        def Download(self,url):
            r = requests.get(url)
            return r.content
        def AddEquity(self,sym):
            pass
        
qb = QuantBook()



In [2]:
actions = json.loads(qb.Download("https://www.dropbox.com/s/psgut932tw77b82/actions.json?dl=1"))
print(actions[0])


def download_data(urls):
    data = []
    for url in urls:
        print(url)
        data.extend(json.loads(qb.Download(url)))
    return mt_dict_to_df(data)

options_cache = download_data(option_urls)
    
print('options_cache:',len(options_cache))

if len(options_cache):


    #options_cache['time'] = options_cache['time'].apply(lambda x: x.to_pydatetime())
    #options_cache['time'] = options_cache['time'].apply(lambda x: datetime(year=1945,month=7,day=6))
    options_cache['time'] = options_cache['time'].dt.to_pydatetime()
    #options_cache['time'] = options_cache['time'].apply(lambda x: datetime.utcfromtimestamp(x))
    options_cache['expiry'] = options_cache['expiry'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date())    
    options_cache['symbol'] = options_cache['symbol'].apply(lambda x: x.split()[0])
    
    
    
    
    #options_cache.set_index(['symbol', 'type', 'strike', 'expiry', 'time'],inplace = True)
    
    print("download type",type(options_cache['time'].values[9]))
    
    ts = options_cache['time'].values[9]

    
    
    ts = options_cache['expiry'].values[9]
    
    print("sorting options_cache")
    options_cache.sort_values(by=['symbol','type','strike','expiry','time'],inplace=True)
    print("finished")
    options_cache.drop_duplicates(inplace=True)
    print("after drop",len(options_cache))
    
    print(type(ts))
    
    

underlying_cache = download_data(underlying_urls)

print('underlying_cache:',len(underlying_cache))

if len(underlying_cache):
    underlying_cache['time'] = underlying_cache['time'].apply(lambda x: x.to_pydatetime())
    
    print('sorting underlying_cache')
    underlying_cache.sort_values(by=['symbol','time'],inplace=True)
    print('finished')
    underlying_cache.drop_duplicates(inplace=True)
    print("after drop:",len(underlying_cache))
    

{'ts': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0]}
https://www.dropbox.com/s/2za6imjbn1i28p8/option_bars.json?dl=1
options_cache: 865056
download type <class 'numpy.datetime64'>
sorting options_cache
finished
after drop 865056
<class 'datetime.date'>
https://www.dropbox.com/s/ll8xa6xb1xhrhv7/underlying_bars.json?dl=1
underlying_cache: 417290
sorting underlying_cache
finished
after drop: 417290


In [3]:
def concat_dfs(df1,df2):
    
    if not len(df1):
        return df2
    
    if not len(df2):
        return df1
                        
    df3 = pd.concat([df1,df2]).drop_duplicates()
    
    
    return df3

def search_underlying_cache(sym,start_time,end_time):
    
    if not len(underlying_cache):
        return pd.DataFrame(),0
    
    equity = qb.AddEquity(sym)
    #df = underlying_cache[ (underlying_cache['symbol'] == str(equity.Symbol.ID)) &
    
    sym = sym + REQUEST_MARKER
    

    
    df = underlying_cache[ (underlying_cache['symbol'] == sym) &
                        (underlying_cache['time'] >= start_time) &
                       (underlying_cache['time'] <= end_time)]
    
    return df,len(df)

def search_options_cache(sym,expiry,strike,spread_type,ts_start,ts_end):
    #symbol	type	strike	expiry	time	close
    
    print("search_options_cache",sym,strike,spread_type)
    
    if not len(options_cache):
        print("search_options_cache",sym,strike,spread_type,"NOT FOUND IN CACHE")
        return pd.DataFrame(),0
    
    sym = sym + REQUEST_MARKER
    
    df = options_cache[ (options_cache['symbol'] == sym)  &
                    (options_cache['type'] == ('Call' if spread_type == 'CALL' else 'Put'))  &
                    (options_cache['strike'] == strike)    &
                    (options_cache['expiry'] == expiry)   &
                    (options_cache['time'] >= ts_start)  &
                    (options_cache['time'] <= ts_end) ]
    
    return df,len(df)

def get_underlying_bars(qb,symv,start_time,end_time):
    
    d,count = search_underlying_cache(symv,start_time,end_time)
    
    if len(d):
        print("found underlying data in cache:",symv)
        if FIND_MISSING:
            print("FIND_MISSING is True, returning empty underlying dataframe")
            return pd.DataFrame()
        return d
    else:
        if not QB:
            return pd.DataFrame()  # Not in cache and cannot look it up.  Probably SPX
    
    print("loading underlying from qc:",symv)
    
    
    equity = qb.AddEquity(symv)
    print("qb.History( {} {} {} {})".format(equity.Symbol, start_time, end_time, Resolution.Minute))
    history = qb.History(equity.Symbol, start_time, end_time, Resolution.Minute)
    
    print(symv,len(history))
    #print("history columns: ",history.columns)
    #print("history index:",list(history.index.names))
    
    
    #drop columns except 'close'.  Index columns are unaffected
    if len(history):
        history.drop(['askclose', 'askhigh', 'asklow', 'askopen', 'asksize', 'bidclose','bidhigh', 'bidlow', 'bidopen', 'bidsize', 'high', 'low','open', 'volume'],axis=1,inplace=True)
        
        history.reset_index(inplace = True)
        
        if history['symbol'][0].startswith('MA'):
            #MAR R9GTMYCXKZ8L
            print('-----------------------debug found underlying symbol--------------',
                  history['symbol'][0])
            
        print("Changing underlying symbol:",history['symbol'][0],'To:',symv + REQUEST_MARKER)
        
        history['symbol'] = symv + REQUEST_MARKER

    
    return history

def get_option_bars(qb,symx,ts,strike_date,spread_type,strikes):
    #print("get_option_bars:",sym,ts,strike_date,spread_type,strikes)
    global opts1,opts2
        

    option_bars = pd.DataFrame()
    counts = []
    

    #print("contracts: ",len(contracts))
    for strike in strikes:
        
        d,count = search_options_cache(symx,strike_date.date(),strike,spread_type,ts,strike_date)
        if len(d):
            if FIND_MISSING:
                print("FIND_MISSSING is True, skipping strike:",strike)
                continue
                
            option_bars = concat_dfs(d,option_bars)
            
            counts.append(count)
            continue
            
        print("missing data:",symx,strike_date.date(),strike,spread_type)
    
        if not FIND_MISSING:
            continue
            
        if not QB:
            continue  #cannot find in cache and unable to search QC
            
        print("loading from qc:",symx,strike_date.date(),strike,spread_type)
        
        #option = qb.AddOption(sym)
        #option.SetFilter(UniverseFunc)
        
        equity = qb.AddEquity(symx, Resolution.Minute)
        contracts = qb.OptionChainProvider.GetOptionContractList(equity.Symbol,ts.date())
        print("len(contrcts)",len(contracts))
        print("ts:",ts)
        lt_strike=0
        eq_strike = None
        gt_strike=10000
        lt_date = datetime.min
        eq_date = None
        gt_date = datetime.max
        for c in contracts:
            if c.ID.OptionRight ==  (0 if spread_type == 'CALL' else 1):
                
                if c.ID.StrikePrice < strike and c.ID.StrikePrice > lt_strike:
                    lt_strike = c.ID.StrikePrice
                if c.ID.StrikePrice > strike and c.ID.StrikePrice < gt_strike:
                    gt_strike = c.ID.StrikePrice
                    
                #if sym=="DKNG":
                #    print(strike,c.ID.StrikePrice)
                    
                if strike == c.ID.StrikePrice:
                    eq_strike = strike
                    #print("found strike",strike,c.ID.Date.date(),strike_date.date())
                    
                    if c.ID.Date < strike_date and c.ID.Date > lt_date:
                        lt_date = c.ID.Date
                        
                    if c.ID.Date > strike_date and c.ID.Date < gt_date:
                        gt_date = c.ID.Date
                    
                    if c.ID.Date.date() == strike_date.date():
                        eq_date = strike_date.date()
                        print("qb.GetOptionHistory",event['sym'],c.ID.Symbol,c.ID.OptionRight,c.ID.Date.date(),c.ID.StrikePrice,c.ID.Market,c.ID.SecurityType)
                        option_history = qb.GetOptionHistory(c, ts,strike_date)
                        d = option_history.GetAllData()
                        
                        d.reset_index(inplace = True)
          

                                
                        d['mid'] = (d['bidclose'] + d['askclose'])/2
                        d.mid.fillna(d.close,inplace=True) # fill any nans with close
                        
                        d.drop(['askhigh', 'asklow', 'bidclose','askclose','close',
                                'askopen', 'asksize','bidhigh', 'bidlow', 
                                'bidopen', 'bidsize', 'high', 'low','open', 'volume'],axis=1,inplace=True}
                        
                        
                        d = movecol(d, 
                             cols_to_move=['symbol','type'], 
                             ref_col='expiry',
                             place='Before')
                        
                        d['symbol'] = symx + REQUEST_MARKER #replace with request symbol. 'r' is to prevent qc from implicit conversion to long symbol
                        
    
                        if len(d):
                            if opts1 is None:
                                opts1 = d.copy()
                            else:
                                if opts2 is None:
                                    opts2 = d.copy()
                                    
                            print("found from qc:",symx,strike_date.date(),strike,spread_type,len(d))
                            
                            option_bars = concat_dfs(d,option_bars)
                                          
                            print("after_concat:",len(option_bars))
                              
                            counts.append(len(d))

        if not eq_strike:
            print("closest strikes to {}: {} {} ".format(strike,lt_strike,gt_strike))
        if not eq_date:
            print("closest dates to {}: {} {}".format(strike_date.date(),lt_date.date(),gt_date.date()))
                    
    if (len(counts) and (counts[0] != counts[-1])):
            print("-------------------------------- Error in lengths: ",counts)
            
    #print("get_option_bars",type(data[0]),data)
        
    return option_bars


In [4]:


class Tick_event:
    event = None
    time = None
    sort_key = 0
    priority = 1  # for sorting, price updates 0, events 1
    underlying_bars = pd.DataFrame()
    option_bars = pd.DataFrame()

    
    def __init__(self,event):

        self.event = event
        self.time = datetime.strptime(event['ts'], '%Y-%m-%dT%H:%M:%S') + timedelta(hours=3)
        self.sort_key = self.time.timestamp()*10 + self.priority
        
    def get_history_data(self):
        #print("get_history_data1")
        
        #if 'sym' not in event.keys():
        #    return [],[]
        
        #if self.event['sym'] != 'MA':
        #    return [],[]

        if all(item in event.keys() for item in 
               ['action','sym','spread_type','strike_date','strikes']):
            
            #if event['sym'] != 'ZM':
            #    return self.underlying_bars,self.option_bars
                
            
            #print('get_history_data:',event)
        
            exp = datetime.strptime(event['strike_date'], '%Y-%m-%d') + timedelta(hours=16)
            if self.event['action'] == 'shorting':
                
                print("shorting: ")
            
                end_time = datetime.strptime(event['strike_date'], '%Y-%m-%d').replace(hour=16, minute = 0, second = 0)
            
                                                      
                self.underlying_bars = get_underlying_bars(qb,
                            event['sym'],
                            self.time,
                            end_time)
                
                #print('type of self.underlying_bars:',type(self.underlying_bars))
            
                self.option_bars = get_option_bars(qb,
                           event['sym'],
                           self.time,
                           end_time,
                           event['spread_type'],
                           event['strikes'])
                
                #print('type of self.option_bars:',type(self.option_bars))
            
                #print('{} shorting: {} {} {} {}'.format(ts,event['sym'],event['spread_type'],exp,event['strikes']))
        if 'stop' in event.keys():
                #print('{} stop: {}'.format(start_time,event['stop']))
            pass
        
        #print("get_history_data2")

        return self.underlying_bars,self.option_bars
    

        
    
    def on_tick(self):
        event = self.event
        
        print("on_tick: ",event)
        
     
    def __repr__(self):
        return str(self.event)


In [5]:
import time



limit_hi = limit_lo + limit_len

option_bars = pd.DataFrame()
underlying_bars = pd.DataFrame()

for i,event in enumerate(actions):
    print('LIMIT = ',i)
    if i >= limit_hi:
        break
    te = Tick_event(event)
    if i >= limit_lo:
        #time.sleep(.5)
        
        u,o = te.get_history_data()
    
        #print("concatting",len(u),len(underlying_bars))
        underlying_bars = concat_dfs(u,underlying_bars)

        #print('history len:',len(underlying_bars))

            
        #if dup_check_underlying(underlying_bars):
        #    print('dupes in underlying_bars')
        #    break
        #print("concat option bars types",type(o),type(option_bars))
        option_bars = concat_dfs(o,option_bars)
        #if len(option_bars):
        #    print("found_option_bars")
        #    break
        
print("")
print("finished")
print("option_bars:",len(option_bars))
print("underlying_bars:",len(underlying_bars))

LIMIT =  0
shorting: 
found underlying data in cache: ZM
search_options_cache ZM 180.0 CALL
search_options_cache ZM 185.0 CALL
LIMIT =  1
shorting: 
found underlying data in cache: PYPL
search_options_cache PYPL 155.0 CALL
search_options_cache PYPL 160.0 CALL
LIMIT =  2
shorting: 
found underlying data in cache: HES
search_options_cache HES 46.0 PUT
search_options_cache HES 44.0 PUT
-------------------------------- Error in lengths:  [1489, 1488]
LIMIT =  3
shorting: 
found underlying data in cache: AAPL
search_options_cache AAPL 315.0 PUT
search_options_cache AAPL 310.0 PUT
LIMIT =  4
shorting: 
found underlying data in cache: GS
search_options_cache GS 185.0 PUT
search_options_cache GS 180.0 PUT
LIMIT =  5
LIMIT =  6
shorting: 
found underlying data in cache: W
search_options_cache W 170.0 CALL
search_options_cache W 175.0 CALL
LIMIT =  7
LIMIT =  8
LIMIT =  9
LIMIT =  10
shorting: 
found underlying data in cache: AMD
search_options_cache AMD 56.0 CALL
search_options_cache AMD 58.0 C

In [ ]:
print("limit_lo =",limit_lo)
print("limit_len=",limit_len)

print("underlying_bars = ",mt_df_to_dict(underlying_bars))


In [ ]:
print("limit_lo =",limit_lo)
print("limit_len=",limit_len)

print("option_bars = ",mt_df_to_dict(option_bars.head(10)))


In [ ]:
option_bars.columns

In [ ]:
print(len(option_bars))
print(len(underlying_bars))

In [ ]:
print(len(opts1))
print(len(opts2))

In [ ]:
underlying_bars

In [ ]:
ts = pd.Timestamp('2014-01-23 00:00:00',tz=None)

In [ ]:
type(ts)

In [ ]:
ts.to_pydatetime()

In [ ]:
option_bars.sort_values(by=['symbol','type','strike','expiry','time'],inplace=True)
underlying_bars.sort_values(by=['symbol','time'],inplace=True)

In [ ]:
print(len(options_cache))
print(len(underlying_cache))

In [ ]:
print(len(option_bars))
print(len(underlying_bars))

In [ ]:
options_cache.head(1000)

In [ ]:
new_dict = mt_df_to_dict(underlying_bars)

In [ ]:
new_df = mt_dict_to_df(new_dict)

In [ ]:
len(new_df)

In [ ]:
for (x1,row1),(x2,row2) in zip(underlying_cache.iterrows(),underlying_bars.iterrows()):
    
    if not row1.equals(row2):
        print(x1,row1,type(row1))
        print(x2,row2)
        break


In [ ]:

options_cache['expiry'] = options_cache['expiry'].apply(lambda x: str(x))

In [ ]:
url = "/home/cc/Dropbox/options_cache_all.json"
with open(url,'w') as outfile:
    json.dump(mt_df_to_dict(options_cache),outfile)

In [ ]:


options_cache.head(3)

In [ ]:
mt_df_to_dict(options_cache.head(3))

In [6]:
class A():
    def __init__(self,val):
        self.val = val
a = A(1)
all_list = []
all_list.append(a)
a = A(2)
for a in all_list:
    print(a.val)


1
